In [1]:
import requests

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import time
import re


start_time = time.time()

# # HTMLから株価を抽出する関数
# def extract_stock_data(html):
#

# # Webページにアクセスしてデータ抽出する関数
# def get_stock_values(driver, url):
#



url = "https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month"

# 待機時間の設定
time_wait = 1 # 待機秒数
# wait = WebDriverWait(driver, 30)

# ブラウザ起動オプション指定
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")

# ブラウザ起動
driver = webdriver.Chrome(options=options)

# 日経新聞のターゲットページにアクセス
driver.get(url)

time.sleep(time_wait)


In [2]:
# グラフのマウスオーバー
elements_to_hover = driver.find_element(By.CSS_SELECTOR, "rect.highcharts-background")
actions = ActionChains(driver)
actions.move_to_element(elements_to_hover).perform()

# グラフの中央をマウスオーバー
elements_to_hover = driver.find_element(By.CSS_SELECTOR, "#highcharts-0 > svg > rect:nth-child(8)")
actions = ActionChains(driver)
actions.move_to_element(elements_to_hover).perform()




In [3]:
# グラフの幅（サイズ）を取得
g_width = elements_to_hover.rect["width"]

# マウスをグラフの右端に移動させる
actions = ActionChains(driver)
actions.move_to_element(elements_to_hover).perform()
actions.move_by_offset(g_width // 2, 0).perform()
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
pattern = r"\d{1,3}(?:,\d{3})*(?:\.\d+)?"

stock_data = []
cnt = 0
date_now = ""
for i in range(g_width - 1):
    actions = ActionChains(driver)
    actions.move_by_offset(-1, 0).perform()
    html = driver.page_source
    # time.sleep(time_wait)
    # print(i) # デバッグ用
    # bs4で日付、始値、高値、安値、終値を取得し、リストに入れる    
    soup = BeautifulSoup(html, "html.parser")
    date = soup.select_one("#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child(1)").text
    date_time = datetime.strptime(date, "%Y/%m/%d")
    if date_now == date:
        continue
    open_price_org = soup.select_one("#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child(2)").text
    open_price_str = re.search(pattern, open_price_org).group(0)
    open_price_str = open_price_str.replace(",", "")
    open_price = float(open_price_str)
    high_price_org = soup.select_one("#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child(3)").text
    high_price_str = re.search(pattern, high_price_org).group(0)
    high_price_str = high_price_str.replace(",", "")
    high_price = float(high_price_str)
    low_price_org = soup.select_one("#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child(4)").text
    low_price = re.search(pattern, low_price_org).group(0)
    close_price_org = soup.select_one("#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child(5)").text
    close_price = re.search(pattern, close_price_org).group(0)
    stock_data.append([date_time, open_price, high_price, low_price, close_price])
    date_now = date
    # print(stock_data) # デバッグ用
    print(f"日付:{stock_data[cnt][0].strftime('%Y年%m月%d日')} 始値:{stock_data[cnt][1]} 高値:{stock_data[cnt][2]} 安値:{stock_data[cnt][3]} 終値:{stock_data[cnt][4]}")
    cnt += 1

# print(stock_data)

driver.quit()

# 実行時間の計算
end_time = time.time()
elapsed_time = end_time - start_time

# 秒数を整数の時,分,秒にする
elapsed_second = int(elapsed_time)
hour = elapsed_second // 3600
minute = (elapsed_second % 3600) // 60
second = elapsed_second % 60

# 実行時間に応じて表示を変える
if hour > 0:
    print(f"実行時間: {hour}時間{minute}分{second}秒")
elif minute > 0:
    print(f"実行時間: {minute}分{second}秒")
else:
    print(f"実行時間: {second}秒")


日付:2024年07月08日 始値:40863.14 高値:41112.24 安値:40,780.7 終値:40,780.7
日付:2024年07月05日 始値:41009.36 高値:41100.13 安値:40,746.89 終値:40,912.37
日付:2024年07月04日 始値:40747.59 高値:40971.23 安値:40,604.15 終値:40,913.65
日付:2024年07月03日 始値:40225.83 高値:40694.21 安値:40,172.85 終値:40,580.76
日付:2024年07月02日 始値:39543.61 高値:40113.48 安値:39,457.62 終値:40,074.69
日付:2024年07月01日 始値:39839.82 高値:39942.69 安値:39,537.1 終値:39,631.06
日付:2024年06月28日 始値:39593.01 高値:39783.7 安値:39,485.73 終値:39,583.08
日付:2024年06月27日 始値:39434.47 高値:39439.24 安値:39,193.51 終値:39,341.54
日付:2024年06月26日 始値:39364.73 高値:39788.63 安値:39,233.56 終値:39,667.07
日付:2024年06月25日 始値:38833.9 高値:39215.07 安値:38,758.07 終値:39,173.15
日付:2024年06月24日 始値:38497.42 高値:38905.47 安値:38,416.07 終値:38,804.65
日付:2024年06月21日 始値:38608.66 高値:38795.68 安値:38,532.13 終値:38,596.47
日付:2024年06月20日 始値:38410.32 高値:38637.62 安値:38,263.08 終値:38,633.02
日付:2024年06月19日 始値:38653.91 高値:38797.97 安値:38,453.41 終値:38,570.76
日付:2024年06月18日 始値:38433.48 高値:38519.02 安値:38,332.89 終値:38,482.11
日付:2024年06月17日 始値:38440.98 高値: